In [1]:
# installing requirements
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.18 which is incompatible.


In [2]:
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
INPUT_SIZE = 37         # state size
NUM_ACTIONS = 4         # the number of possible actions


FC1_UNITS = 128
SEED = 13

In [4]:
import torch
from dqn_agent import Agent
from model import QNetwork
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#create the agent
agent = Agent(action_size=NUM_ACTIONS,
              seed=SEED,
              qnetwork_local = QNetwork(INPUT_SIZE, NUM_ACTIONS, SEED, FC1_UNITS).to(device),
              qnetwork_target = QNetwork(INPUT_SIZE, NUM_ACTIONS, SEED, FC1_UNITS).to(device))

agent.qnetwork_local.load_state_dict(torch.load('model.pt'))
agent.qnetwork_local.eval()

QNetwork(
  (fc1): Linear(in_features=37, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [5]:
for i in range(10):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment, I set train_mode = true for speed
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = agent.act(state)                 # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    print("Episode: {} Score: {}".format(i, score))

Episode: 0 Score: 15.0
Episode: 1 Score: 19.0
Episode: 2 Score: 16.0
Episode: 3 Score: 18.0
Episode: 4 Score: 16.0
Episode: 5 Score: 18.0
Episode: 6 Score: 18.0
Episode: 7 Score: 14.0
Episode: 8 Score: 21.0
Episode: 9 Score: 19.0


In [6]:
env.close()